In [1]:
import cv2
import numpy as np
import face_recognition
import os
from os import makedirs, listdir
from os.path import isdir, isfile, join

# 1. Collecting Dataset

In [2]:
# https://blog.naver.com/chandong83/221695462391

# function for consecutive capture
def detect_face(frame, classifier):
    grayscale_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = classifier.detectMultiScale(grayscale_frame, 1.3, 3)   # image, image_scale, max_faces
    if len(faces) == 0:
        return None
    for(x,y,w,h) in faces:
        margin = 0.1 # 배경 비율
        x = int (x - w * margin) # x 좌표를 왼쪽으로 이동
        y = int (y - h * margin) # y 좌표를 위로 이동
        w = int (w * (1 + margin * 2)) # 너비를 늘림
        h = int (h * (1 + margin * 2)) # 높이를 늘림
        cropped_face = frame[y:y+h, x:x+w]
    
    return cropped_face


In [3]:
# https://blog.naver.com/chandong83/221695462391

def capture_face(user_name):
    classifier = cv2.CascadeClassifier("data/haarcascade_frontalface_default.xml")
    if not isdir(f"data/sub_data/{user_name}"):
        makedirs(f"data/sub_data/{user_name}")
    capture= cv2.VideoCapture(0)
    count = 0
    
    while True:
        result, frame = capture.read()
        if detect_face(frame, classifier) is not None:
            count+=1
            face = cv2.resize(detect_face(frame, classifier),(200,200))
            # face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            cv2.imwrite(f"data/sub_data/{user_name}/{user_name}-{count}.png",face)
            cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2)
            cv2.imshow('Face Cropper',face)
        else:
            print("Face not Found")
            pass
        # Press Enter or get 50images to break
        if cv2.waitKey(1)==13 or count==50:
            break

    capture.release()
    cv2.destroyAllWindows()
    print('Colleting Samples Complete!!!')

In [4]:
# capture_face("DDD")

In [5]:
path = 'data/sub_data'
user_list = [item for item in os.listdir (path) if os.path.isdir (os.path.join (path, item))]

In [6]:
user_list

['calmdownman',
 'former_cafe_owner',
 'gaon',
 'kane',
 'lee',
 'pearl',
 'seung_yeon',
 'sung_jun']

# 2. Train Model

In [7]:
import math
from sklearn import neighbors
import os
import os.path
import pickle
from PIL import Image, ImageDraw
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder

In [8]:
# https://github.com/ageitgey/face_recognition/blob/master/examples/face_recognition_knn.py

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}


def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo='ball_tree', verbose=False):
    """
    Trains a k-nearest neighbors classifier for face recognition.

    :param train_dir: directory that contains a sub-directory for each known person, with its name.

     (View in source code to see train_dir example tree structure)

     Structure:
        <train_dir>/
        ├── <person1>/
        │   ├── <somename1>.jpeg
        │   ├── <somename2>.jpeg
        │   ├── ...
        ├── <person2>/
        │   ├── <somename1>.jpeg
        │   └── <somename2>.jpeg
        └── ...

    :param model_save_path: (optional) path to save model on disk
    :param n_neighbors: (optional) number of neighbors to weigh in classification. Chosen automatically if not specified
    :param knn_algo: (optional) underlying data structure to support knn.default is ball_tree
    :param verbose: verbosity of training
    :return: returns knn classifier that was trained on the given data.
    """
    X = []
    y = []

    # Loop through each person in the training set
    for class_dir in os.listdir(train_dir):
        if not os.path.isdir(os.path.join(train_dir, class_dir)):
            continue

        # Loop through each training image for the current person
        for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
            image = face_recognition.load_image_file(img_path)
            face_bounding_boxes = face_recognition.face_locations(image)

            if len(face_bounding_boxes) != 1:
                # If there are no people (or too many people) in a training image, skip the image.
                if verbose:
                    print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
            else:
                # Add face encoding for current image to the training set
                X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                y.append(class_dir)

    # Determine how many neighbors to use for weighting in the KNN classifier
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically:", n_neighbors)

    # Create and train the KNN classifier
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)

    # Save the trained KNN classifier
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf


# 3. Predict

In [9]:
def predict(X_img_path, knn_clf=None, model_path=None, distance_threshold=0.6):
    """
    Recognizes faces in given image using a trained KNN classifier

    :param X_img_path: path to image to be recognized
    :param knn_clf: (optional) a knn classifier object. if not specified, model_save_path must be specified.
    :param model_path: (optional) path to a pickled knn classifier. if not specified, model_save_path must be knn_clf.
    :param distance_threshold: (optional) distance threshold for face classification. the larger it is, the more chance
           of mis-classifying an unknown person as a known one.
    :return: a list of names and face locations for the recognized faces in the image: [(name, bounding box), ...].
        For faces of unrecognized persons, the name 'unknown' will be returned.
    """
    if not os.path.isfile(X_img_path) or os.path.splitext(X_img_path)[1][1:] not in ALLOWED_EXTENSIONS:
        raise Exception("Invalid image path: {}".format(X_img_path))

    if knn_clf is None and model_path is None:
        raise Exception("Must supply knn classifier either thourgh knn_clf or model_path")

    # Load a trained KNN model (if one was passed in)
    if knn_clf is None:
        with open(model_path, 'rb') as f:
            knn_clf = pickle.load(f)

    # Load image file and find face locations
    X_img = face_recognition.load_image_file(X_img_path)
    X_face_locations = face_recognition.face_locations(X_img)

    # If no faces are found in the image, return an empty result.
    if len(X_face_locations) == 0:
        return []

    # Find encodings for faces in the test iamge
    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)

    # Use the KNN model to find the best matches for the test face
    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]

    # Predict classes and remove classifications that aren't within the threshold
    return [(pred, loc) if rec else ("@unknown", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]


In [10]:
def show_prediction_labels_on_image(img_path, predictions):
    """
    Shows the face recognition results visually.

    :param img_path: path to image to be recognized
    :param predictions: results of the predict function
    :return:
    """
    pil_image = Image.open(img_path).convert("RGB")
    draw = ImageDraw.Draw(pil_image)
    for name, (top, right, bottom, left) in predictions:
        # Draw a box around the face using the Pillow module
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))
        # There's a bug in Pillow where it blows up with non-UTF-8 text
        # when using the default bitmap font
        name = name.encode("UTF-8")
        # Draw a label with a name below the face
        text_width, text_height = draw.textsize(name)
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))
        print(f'{name} Detected, TODO: Save History....')
    # Remove the drawing library from memory as per the Pillow docs
    del draw

    # Display the resulting image
#     pil_image.show()
    display(pil_image)

In [11]:
def save_prediction_to_database(predictions):
    for name, (top, right, bottom, left) in predictions:
        print(f'{name} Detected, TODO: Save History....')
    

In [12]:
def capture_visitor_face():
    classifier = cv2.CascadeClassifier("data/haarcascade_frontalface_default.xml")
    capture= cv2.VideoCapture(0)
    count = 0
    
    while True:
        result, frame = capture.read()
        cv2.imshow("VideoFrame", frame)
        cropped_visitor_face = detect_face(frame, classifier)
        if cropped_visitor_face is not None:
            cv2.imwrite(f'data/test_data/visitor.png', cropped_visitor_face)
            break
        else:
            print("Visitor Face not found ")
        if cv2.waitKey(1)==13:
            break

    capture.release()
    cv2.destroyAllWindows()
    print('visitor passed door')

In [13]:
# capture_visitor_face()

# 4. Save history to database

In [14]:
import pymysql
import time
import datetime

In [ ]:
host_, user_, password_, db_ = ['IP_ADDRESS', 'MYSQL_ID', 'PASS_WORD', 'DB']

In [ ]:
# host_, user_, password_, db_ = input().split()

In [28]:
# Insert Example 1

def save_prediction_to_database_2(predictions):
    conn = pymysql.connect(host=host_, user=user_, 
                           password=password_, db=db_, charset='utf8')
    user_id = "@unknown"
    for name, (top, right, bottom, left) in predictions:
        if(not name.startswith('@')):
            user_id = name
            print(f'{name} Detected')
            break
    if(user_id != "@unknown"):
        try:
            curs = conn.cursor()
            sql = "INSERT INTO visitor_info VALUES (%s, %s)"
            ts = time.time()
            timestamp = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
            val = (user_id, timestamp)
            curs.execute(sql, val)
            conn.commit()
        finally: 
            conn.close()

In [26]:
def main():
    # STEP 1: Train the KNN classifier and save it to disk
    # Once the model is trained and saved, you can skip this step next time.
#     print("Training KNN classifier...")
#     classifier = train("data/sub_data", model_save_path="data/trained_knn_model.clf", n_neighbors=2)
#     print("Training complete!")

    # STEP 2: Using the trained classifier, make predictions for unknown images
    #  Take temp picture
    counts = 0 
    while True:
        if(counts >= 5):
            break
        capture_visitor_face()
        predictions = predict("data/test_data/visitor.png", model_path="data/trained_knn_model.clf")
        # Print results on the console
        # TODO: 화면에 표시하지 않고 서버로 보낼 것
        save_prediction_to_database_2(predictions)
        time.sleep(3)
        counts += 1 

In [27]:
main()

visitor passed door
lee Detected
visitor passed door
lee Detected
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found

Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not

Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
visitor passed door
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not found 
Visitor Face not fou